In [1]:
#Importing the necessary libraries

In [2]:
import cv2
from object_detector import*
import numpy as np
import cv2.aruco as aruco

In [3]:
pip show object-detection

Name: object-detection
Version: 0.0.3
Summary: A package build from Tensorflow's object detection API.
Home-page: https://github.com/brandonschabell/models/tree/master/research/object_detection
Author: Brandon Schabell
Author-email: brandonschabell@gmail.com
License: Apache License 2.0
Location: c:\users\dell\anaconda3\envs\dl_env\lib\site-packages
Requires: matplotlib, pillow, jupyter, lxml, tensorflow, contextlib2, Cython
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [3]:
#OpenCV allows a straightforward interface to capture live stream with the camera (webcam). 
#It converts video into grayscale and display it.
cap=cv2.VideoCapture(0)
#The cap.read() returns a boolean value(True/False).It will return True, if the frame is read correctly.


#cap.set(cv2.CAP_PROP_FRAME_WIDTH,1280)
#cap.set(cv2.CAP_PROP_FRAME_WIDTH,720)

In [4]:
#What is Auroc Marker?

#ArUco stands for Augmented Reality University of Cordoba. That is where it was developed in Spain.

#An aruco marker is a fiducial marker that is placed on the object or scene being imaged. 
#It is a binary square with black background and boundaries and a white generated pattern within it that uniquely identifies it.


#The black boundary helps making their detection easier. 
#They can be generated in a variety of sizes. 
#The size is chosen based on the object size and the scene, for a successful detection.


#We are using Aurco marker as our reference object

In [5]:
#for this research we have selected the Aurco marker which is 5 x 5 cm

In [6]:
parameters = cv2.aruco.DetectorParameters_create()
#cv2.aruco.detectMarkers to detect tags for the current arucoDict in the input image


aurco_dict=cv2.aruco.Dictionary_get(cv2.aruco.DICT_5X5_50)
#once detected we are loading the it using the cv2.aruco.Dictionary_get

In [7]:
# Load Object Detector
detector = HomogeneousBgDetector()#detection for the other objects 

In [8]:
#Stage 1

In [9]:
while(True):
    _,img1 = cap.read()
    # Get Aruco marker
    corners, _, _ = cv2.aruco.detectMarkers(img1, aurco_dict, parameters=parameters)#We need to detect the aruco marker in the photo and only obtain the corners.
    # Draw polygon around the marker
    if corners:
        int_corners = np.int0(corners)#Signed integer type
        #cv2.polylines(img1, int_corners, True, (0, 255, 0), 5)#We draw a polygon around the marker to make sure we have identified it correctly.
        # Aruco Perimeter : perimeter formula : 4 x (side)
        
        #We know that the Aruco marker used is a square with 5 cm per side so we have a perimeter of 20 cm
        aruco_perimeter = cv2.arcLength(corners[0], True)
        # Pixel to cm ratio
        pixel_cm_ratio = aruco_perimeter / 20
        contours = detector.detect_objects(img1)#A contour is the line which defines a form or edge - an outline.
        # Draw objects boundaries
        
        #Within the for loop relating to the contours of the objects, among other things, 
        #also two parameters h and w that correspond to the height and width are extracted.
        
        #Applying the calculation of the ratio to these two variables we obtain the centimeters.
        for cnt in contours:
        # Get rect
            rect = cv2.minAreaRect(cnt)
            (x, y), (w, h), angle = rect
            # Get Width and Height of the Objects by applying the Ratio pixel to cm
            
            #Convert pixels to cm of objects
            object_width = w / pixel_cm_ratio
            object_height = h / pixel_cm_ratio
            
            #display the rectangle
            box = cv2.boxPoints(rect)
            box = np.int0(box)#converting the float to integer numbers
            #Draw polygon
            cv2.polylines(img1,[box],True,(0,0,255),2)
            
            #we just have to print the result on the screen
            cv2.putText(img1, "Width {} cm".format(round(object_width, 1)), (int(x - 100), int(y - 20)), cv2.FONT_HERSHEY_PLAIN, 2, (100, 200, 0), 2)
            cv2.putText(img1, "Height {} cm".format(round(object_height, 1)), (int(x - 100), int(y + 15)), cv2.FONT_HERSHEY_PLAIN, 2, (100, 200, 0), 2)

    
    cv2.imshow('image', img1) 
  
    # Maintain output window utill
    # user presses a key
    key=cv2.waitKey(1)
    if key == 27:
        break
cap.release()
# Destroying present windows on screen
cv2.destroyAllWindows() 

In [10]:
#Stage 2

In [11]:
#focous only on one object instead of multiple objects

In [12]:
#OpenCV allows a straightforward interface to capture live stream with the camera (webcam). 
#It converts video into grayscale and display it.
cap=cv2.VideoCapture(0)
#The cap.read() returns a boolean value(True/False).It will return True, if the frame is read correctly.


#cap.set(cv2.CAP_PROP_FRAME_WIDTH,1280)
#cap.set(cv2.CAP_PROP_FRAME_WIDTH,720)

In [13]:
 # Create point matrix get coordinates of mouse click on image
point_matrix = np.zeros((2,2),np.int)
print(point_matrix)

[[0 0]
 [0 0]]


In [14]:
counter = 0
def mousePoints(event,x,y,flags,params):
    global counter
    # Left button mouse click event opencv
    if event == cv2.EVENT_LBUTTONDOWN:
        point_matrix[counter] = x,y
        counter = counter + 1
 
 
while True:
    _,img = cap.read()
    for x in range (0,2):
        cv2.circle(img,(point_matrix[x][0],point_matrix[x][1]),3,(0,255,0),cv2.FILLED)
 
    if counter == 2:
        starting_x = point_matrix[0][0]
        starting_y = point_matrix[0][1]
 
        ending_x = point_matrix[1][0]
        ending_y = point_matrix[1][1]
        # Draw rectangle for area of interest
        cv2.rectangle(img, (starting_x, starting_y), (ending_x, ending_y), (0, 255, 0), 3)
 
        # Cropping image
        img_cropped = img[starting_y:ending_y, starting_x:ending_x]
        corners, _, _ = cv2.aruco.detectMarkers(img_cropped, aurco_dict, parameters=parameters)
        # Draw polygon around the marker
        if corners:
            int_corners = np.int0(corners)
            cv2.polylines(img_cropped, int_corners, True, (0, 255, 0), 5)
            # Aruco Perimeter
            aruco_perimeter = cv2.arcLength(corners[0], True)
            # Pixel to cm ratio
            pixel_cm_ratio = aruco_perimeter / 20
            contours = detector.detect_objects(img_cropped)
            # Draw objects boundaries
            for cnt in contours:
            # Get rect
                rect = cv2.minAreaRect(cnt)
                (x, y), (w, h), angle = rect
                # Get Width and Height of the Objects by applying the Ratio pixel to cm
                object_width = w / pixel_cm_ratio
                object_height = h / pixel_cm_ratio

                cv2.putText(img_cropped, "Width {} cm".format(round(object_width, 1)), (int(x - 100), int(y - 20)), cv2.FONT_HERSHEY_PLAIN, 2, (100, 200, 0), 2)
                cv2.putText(img_cropped, "Height {} cm".format(round(object_height, 1)), (int(x - 100), int(y + 15)), cv2.FONT_HERSHEY_PLAIN, 2, (100, 200, 0), 2)

        cv2.imshow("ROI", img_cropped)
 
    # Showing original image
    cv2.imshow("Original Image ", img)
    # Mouse click event on original image
    cv2.setMouseCallback("Original Image ", mousePoints)
    # Printing updated point matrix
    #print(point_matrix)
    # Refreshing window all time
    key=cv2.waitKey(1)
    if key == 27:
        break
cap.release()
# Destroying present windows on screen
cv2.destroyAllWindows() 

In [15]:
#Stage 3

In [16]:
#Trying to hide the aurco marker using some sticker

In [17]:
#OpenCV allows a straightforward interface to capture live stream with the camera (webcam). 
#It converts video into grayscale and display it.
cap=cv2.VideoCapture(0)
#The cap.read() returns a boolean value(True/False).It will return True, if the frame is read correctly.


#cap.set(cv2.CAP_PROP_FRAME_WIDTH,1280)
#cap.set(cv2.CAP_PROP_FRAME_WIDTH,720)

In [18]:
counter = 0
def mousePoints(event,x,y,flags,params):
    global counter
    # Left button mouse click event opencv
    if event == cv2.EVENT_LBUTTONDOWN:
        point_matrix[counter] = x,y
        counter = counter + 1
 
# Read image
#img = cv2.imread("D:\\Gloify\\12-11-2021\\1.jpg")
 
while True:
        #_,img = cap.read()
        _,img_cropped = cap.read()
    #for x in range (0,2):
    #    cv2.circle(img,(point_matrix[x][0],point_matrix[x][1]),3,(0,255,0),cv2.FILLED)
 
   # if counter == 2:
        #starting_x = point_matrix[0][0]
        #starting_y = point_matrix[0][1]
 
        #ending_x = point_matrix[1][0]
        #ending_y = point_matrix[1][1]
        # Draw rectangle for area of interest
        #cv2.rectangle(img, (starting_x, starting_y), (ending_x, ending_y), (0, 255, 0), 3)
 
        # Cropping image
        #img_cropped = img[starting_y:ending_y, starting_x:ending_x]
        corners, ids, _ = cv2.aruco.detectMarkers(img_cropped, aurco_dict, parameters=parameters)
        #-----------------------------------------------------------------------------------------#
        if np.all(ids != None):
            display = aruco.drawDetectedMarkers(img_cropped, corners)
            x1 = (corners[0][0][0][0], corners[0][0][0][1]) 
            x2 = (corners[0][0][1][0], corners[0][0][1][1]) 
            x3 = (corners[0][0][2][0], corners[0][0][2][1]) 
            x4 = (corners[0][0][3][0], corners[0][0][3][1])  

            im_dst = img_cropped 
            im_src = cv2.imread("D:\\Gloify\\15-11-2021\\Gloify.png")
            size = im_src.shape
            pts_dst = np.array([x1,x2,x3,x4])
            pts_src = np.array(
                           [
                            [0,0],
                            [size[1] - 1, 0],
                            [size[1] - 1, size[0] -1],
                            [0, size[0] - 1 ]
                            ],dtype=float
                           );


            h, status = cv2.findHomography(pts_src, pts_dst)
            temp = cv2.warpPerspective(im_src, h, (im_dst.shape[1],im_dst.shape[0])) 
            cv2.fillConvexPoly(im_dst, pts_dst.astype(int), 0, 16);
            im_dst = im_dst + temp  
            cv2.imshow('Display',im_dst)
            key=cv2.waitKey(1)
            if key == 27:
                break
        else:
            display = img_cropped
            cv2.imshow('Display',display)
            key=cv2.waitKey(1)
            if key == 27:
                break
        #------------------------------------------------------------------------------------------------------#
        # Draw polygon around the marker
        #==================================================================================================
        
    #=========================================================================================
 
    # Showing original image
        #cv2.imshow("Original Image ", img)
        # Mouse click event on original image
        #cv2.setMouseCallback("Original Image ", mousePoints)
        # Printing updated point matrix
        #print(point_matrix)
        # Refreshing window all time
    #key=cv2.waitKey(1)
        #if key == 27:
            #break
cap.release()
# Destroying present windows on screen
cv2.destroyAllWindows() 

In [ ]:
#--------------------------------------------------------END-----------------------------------------------#